In [1]:
!git clone https://gitee.com/kjj0518/Expression.git

Cloning into 'Expression'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


In [2]:
cd Expression/models

/content/Expression/models


In [3]:
!gdown https://drive.google.com/uc?id=1wR7kCZH7cPlkA_XhTZqMNC3u2i8T50Oh

Downloading...
From: https://drive.google.com/uc?id=1wR7kCZH7cPlkA_XhTZqMNC3u2i8T50Oh
To: /content/Expression/models/model.pth
100% 114M/114M [00:01<00:00, 97.7MB/s]


In [4]:
cd ..

/content/Expression


In [5]:
!pip install efficientnet_pytorch==0.6

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.0-py3-none-any.whl size=12411 sha256=ab01acfa67e2ee73ba1ea6fbbd153b57c85aebbc319c3f56fe1eeb9232a007eb
  Stored in directory: /root/.cache/pip/wheels/33/3e/35/e54695cf32698610843bb6629152efd7b558bb6472ff4b0bdd
Successfully built efficientnet-pytorch


In [6]:
import argparse
import cv2
import torch
from vision.ssd.config.fd_config import define_img_size
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
import os
import PIL
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
from utils import visualize_cam, Normalize
from gradcam import GradCAM, GradCAMpp
from torchvision.transforms import ToPILImage
parser = argparse.ArgumentParser(description='detect_video')

parser.add_argument('--input_size', default=640, type=int,
                    help='define network input size,default optional value 128/160/320/480/640/1280')
parser.add_argument('--threshold', default=0.7, type=float,
                    help='score threshold')
parser.add_argument('--candidate_size', default=1000, type=int,
                    help='nms candidate size')
parser.add_argument('--path', default="imgs", type=str,
                    help='imgs dir')
parser.add_argument('--test_device', default="cuda:0", type=str,
                    help='cuda:0 or cpu')
parser.add_argument('--video_path', default=".MP4", type=str,
                    help='path of video')
args = parser.parse_known_args()[0]

input_img_size = args.input_size
define_img_size(input_img_size)
transform =transforms.Compose([transforms.Resize([100, 100]),transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

from vision.ssd.mb_tiny_RFB_fd import create_Mb_Tiny_RFB_fd, create_Mb_Tiny_RFB_fd_predictor
from vision.utils.misc import Timer

# cap = cv2.VideoCapture(args.video_path)  # capture from video

class_id={'Surprise':0,'Fear':1,'Disgust':2,'Happy':3,'Sad':4,'Angry':5,'Neutral':6}
express=list(class_id)
test_device = args.test_device
candidate_size = args.candidate_size
threshold = args.threshold


if input_img_size>= 640 :
    model_path = "models/face-detect-640.pth"
else:
    model_path = "models/face-detect-320.pth"
net = create_Mb_Tiny_RFB_fd(2, is_test=True, device=test_device)
predictor = create_Mb_Tiny_RFB_fd_predictor(net, candidate_size=candidate_size, device=test_device)
plt.ion()
net.load(model_path)
ExpressionNet=torch.load('models/model.pth').to(test_device)
ExpressionNet.eval()
timer = Timer()
sum = 0

ind = 0.1+0.6*np.arange(len(express))    # the x locations for the groups
width = 0.4       # the width of the bars: can also be len(x) sequence
color_list = ['darkgreen','darkviolet','deepskyblue','limegreen','blue','red','gold']

cam_dict = dict()

efficientnet_model_dict = dict(type='efficientnet', arch=ExpressionNet, layer_name='_conv_head', input_size=(100, 100))
efficientnet_gradcam = GradCAM(efficientnet_model_dict, True)
efficientnet_gradcampp = GradCAMpp(efficientnet_model_dict, True)
cam_dict['efficientnet'] = [efficientnet_gradcam, efficientnet_gradcampp]

priors nums:17640


/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.padding.ZeroPad2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attri

saliency_map size : torch.Size([3, 3])
saliency_map size : torch.Size([3, 3])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [7]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      //div_out.style.backgroundColor='green'
      
      img = document.createElement('img');
      div_out.appendChild(img);

      div_right = document.createElement('div');
      div_out.appendChild(div_right);
      //div_right.style.backgroundColor='yellow'

      img1 = document.createElement('img');
      div_right.appendChild(img1);
      img2 = document.createElement('img');
      div_right.appendChild(img2);
      img3 = document.createElement('img');
      div_right.appendChild(img3);
      img4 = document.createElement('img');
      div_right.appendChild(img4);
      
      
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64,num){
      if (num==0){img.src = "data:image/jpg;base64," + imgb64;}
      if (num==1){img1.src = "data:image/jpg;base64," + imgb64;}
      if (num==2){img2.src = "data:image/jpg;base64," + imgb64;}
      if (num==3){img3.src = "data:image/jpg;base64," + imgb64;}
      if (num==4){img4.src = "data:image/jpg;base64," + imgb64;}
    }
    

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

def png2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'png')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [11]:
from torchvision.transforms import ToPILImage
from google.colab.patches import cv2_imshow
# classify_result = []
# cam = []
# canvas = plt.figure().canvas
# VideoCapture()
# eval_js('create()')

<IPython.core.display.Javascript object>

<Figure size 432x288 with 0 Axes>

In [ ]:
from torchvision.transforms import ToPILImage
from google.colab.patches import cv2_imshow
classify_result = []
cam = []
canvas = plt.figure().canvas
VideoCapture()
eval_js('create()')
while True:
    classify_result.clear()
    cam.clear()
    byte = eval_js('capture()')
    image = byte2image(byte)
    boxes, labels, probs = predictor.predict(image, candidate_size / 2, threshold)    
    for i in range(0,boxes.size(0),1):
        box = boxes[i, :]
        x = box[0] + box[2]
        x = int(x / 2)
        y = box[1] + box[3]
        y = int(y / 2)
        w =box[2]-box[0]+box[3]-box[1]
        w=w.int().item()//4
        face = image[max(y-w,0):y+w,x-w:x+w]
        face = Image.fromarray(face)
        face = transform(face)
        out = ExpressionNet(face.unsqueeze(0).to(test_device))
        expression = torch.argmax(out).item()
        score = torch.nn.functional.softmax(out,dim=1).squeeze()
        cv2.rectangle(image, (x-w,y-w), (x+w, y+w), (0, 255, 0), 4)
        cv2.putText(image, express[expression],
                    (box[0], box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,  # font scale
                    (255, 0, 0),
                    2)  # line type
        for gradcam, gradcam_pp in cam_dict.values():
          mask_pp, _ = gradcam_pp(face.unsqueeze(0).to(test_device))
          heatmap_pp, result_pp = visualize_cam(mask_pp.cpu(), face.unsqueeze(0)*0.5+0.5)
        cam.append(np.asarray(ToPILImage()(result_pp).resize((256,256))))
        plt.clf()
        plt.title("Classification results ", fontsize=15)
        plt.xlabel(" Expression Category ", fontsize=12)
        plt.ylabel(" Classification Score ", fontsize=12)
        plt.xticks(ind, express, rotation=15, fontsize=16)
        plt.ylim([0,1])
        for i in range(len(express)):
            plt.bar(ind[i], score.data.cpu().numpy()[i], width, color=color_list[i])
        buffer = io.BytesIO()
        canvas.print_png(buffer)
        data = buffer.getvalue()
        buffer.write(data)
        buffer.seek(0)
        classify_result.append(b64encode(buffer.read()).decode('utf-8'))
    eval_js('showimg("{0}",0)'.format(image2byte(image)))
    for i in range(len(classify_result)):
      eval_js('showimg("{0}","{1}")'.format(classify_result[i],i*2+1))
      eval_js('showimg("{0}","{1}")'.format(image2byte(cam[i]),i*2+2))


<IPython.core.display.Javascript object>

Inference time:  0.011228


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Inference time:  0.010193
Inference time:  0.011057
